### En este módulo se pasa la hoja de excel a un dataframe. Finalmente se retorna un csv con los datos limpios.
#### La reestructuración de los datos incluye: 
* Incluir una columna que contabiliza los semestres que ha cursado el alumno
* Seleccionar a los alumnos que han cursado los cursos del major.
* Eliminar los casos de convalidaciones
* Realizar filtro de buen y mal rendimiento

In [1]:
import pandas as pd

data = pd.read_excel('notas_alumnos.xlsx', index_col=None, dtype={'ID': int, 'AÑO ADMISIÓN': int, 'MAJOR SELECCIONADO': str, 
                                                                  'SIGLA': str, 'NOMBRE CURSO': str, 'NOTA FINAL': float,
                                                                  'AÑO': int, 'SEMESTRE': int, 'PPA GLOBAL': float})

df = data[['ID', 'SIGLA', 'NOTA FINAL', 'AÑO', 'SEMESTRE', 'PPA GLOBAL']]

In [2]:
# Agrupar los datos por alumno y agregar una columna que indica el número de semestre de la carrera
stg = df.groupby('ID')
sem_array = []
for student, group in stg:
    sem = 0
    year = 0
    path = 0
    for row in group.itertuples():
        if row.AÑO > year or row.SEMESTRE > sem:
            year = row.AÑO
            sem = row.SEMESTRE
            path += 1
        sem_array.append(path)

df.insert(4, 'DURACIÓN', sem_array, True)

In [3]:
# Seleccionar los alumnos que completaron el major (realizar los cursos mínimos)
def students_selection(stgroups, courses, options=[]):
    students = []
    for student, group in stgroups:
        done = group["SIGLA"].tolist()
        valid = False
        if len(options) > 0:
            for course in options:
                if course in done:
                    done.remove(course)
                    valid = True
        else:
            valid = True
        if set(courses) == set(done) and valid:
            students.append(student)   
    return students

In [4]:
# Filtrar alumnos que han convalidado +3 cursos el primer semestre
def filter_conv(df):
    students = df.groupby(['ID'])
    conv_students = []
    filter_df = df
    for student, group in students:
        courses = group["SIGLA"].tolist()
        semestres = group["DURACIÓN"].tolist()
        if semestres[0] == semestres[1] and semestres[0] == semestres[2]:
            conv_students.append(student)
    for stud in conv_students:
        filter_df = df[df.ID != stud]
        filter_df = filter_df.reset_index(drop=True)
    return filter_df

In [5]:
# Leer los majors desde el archivo
import json

with open('majors.json', 'r', encoding='utf-8') as json_majors:
    j = json_majors.read()
    majors = json.loads(j)

# Para cada major generar un archivo csv con los datos de sus alumnos.
# Los alumnos deben tener aprobados todos los cursos del major.
# Se generan 3 csv por major, uno general, uno de buen rendimiento y otro de mal rend.
for major in majors:
    df_base = df.loc[df['SIGLA'].isin(majors[major])]
    df_approved = df_base[df_base['NOTA FINAL'] >= 4]
    student_groups = df_approved.groupby(['ID'])
    major_students = students_selection(student_groups, majors[major])
    df_major = df_approved.loc[df_base['ID'].isin(major_students)]
    df_major = filter_conv(df_major)
    df_major.to_csv("data/{}.csv".format(major))
    good = df_major[df_major['PPA GLOBAL'] >= 5.5]
    good.to_csv("data/{}-G.csv".format(major))
    bad = df_major[df_major['PPA GLOBAL'] < 5]
    bad.to_csv("data/{}-B.csv".format(major))


In [6]:
# Generar CSV de los clusters
cs = [[ 5, 24, 34, 43, 44, 48, 56, 61, 67, 69, 75, 82, 88, 90],
      [ 4,  9, 10, 35, 39, 41, 46, 47, 49, 50, 51, 52, 57, 58, 59, 60, 63,
       64, 65, 66, 68, 70, 72, 76, 78, 79, 84, 85, 86],
      [ 0,  2,  3,  8, 11, 12, 13, 14, 15, 16, 17, 19, 25, 26, 27, 28, 29,
       30, 31, 32, 36, 37, 38, 42, 45, 53, 54, 55, 87, 89],
      [ 1,  6,  7, 18, 20, 21, 22, 23, 33, 40, 62, 71, 73, 74, 77, 80, 81,
       83, 91]]
for i in range(0,len(cs)):
    major= 'Computación 2013'
    df_base = df.loc[df['SIGLA'].isin(majors[major])]
    df_approved = df_base[df_base['NOTA FINAL'] >= 4]
    student_groups = df_approved.groupby(['ID'])
    cluster_students = cs[i]
    df_major = df_approved.loc[df_base['ID'].isin(major_students)]
    df_major = filter_conv(df_major)
    df_major.to_csv("data/{} c{}.csv".format(major, i))